In [2]:
import vaex
import json
import pandas as pd
from tqdm import tqdm
from src.utils import read_yaml
from src.data.make_dataset import explode_str_column, get_new_index, safe_loc
from src.data.utils import  process_minute_level_pandas

In [3]:
public_participants = read_yaml("/projects/bdata/datasets/gatesfoundation/processed/COVID_Signals/covidsignals_synapse_consent.yaml")
SURVEY_REGEX = "/projects/bdata/datasets/gatesfoundation/raw/COVID_Signals/*/all_other_datafiles/survey/daily/*.parquet"
daily_surveys = vaex.open(SURVEY_REGEX).to_pandas_df()
daily_surveys["participant_id"] = daily_surveys["id_participant_external"]

In [6]:
daily_surveys["timestamp"] = daily_surveys["tm_start"]
daily_surveys_consented = daily_surveys[daily_surveys["id_participant_external"].astype(int).isin(set(public_participants))]
daily_surveys_consented.to_csv("/projects/bdata/datasets/gatesfoundation/processed/COVID_Signals/daily_surveys.csv",index=False)

In [8]:
daily_surveys_consented['id_participant_external'].head(10)

0     019616
2     056076
3     056076
4     082767
5     103308
6     109987
7     127041
8     139209
9     149059
10    169793
Name: id_participant_external, dtype: object

In [5]:
WEEKLY_SURVEY_REGEX = "/projects/bdata/datasets/gatesfoundation/raw/COVID_Signals/*/all_other_datafiles/survey/weekly/*.parquet"
weekly_surveys = vaex.open(WEEKLY_SURVEY_REGEX).to_pandas_df()
weekly_surveys_consented = weekly_surveys[weekly_surveys["id_participant_external"].astype(int).isin(set(public_participants))]
weekly_surveys["participant_id"] = weekly_surveys["id_participant_external"]
weekly_surveys_consented.to_csv("/projects/bdata/datasets/gatesfoundation/processed/COVID_Signals/weekly_surveys.csv",index=False)

In [ ]:
pd.to_datetime(weekly_surveys_consented["date_of_test_kit_completion"].str.split("|").str[-1],errors="coerce")

1      2020-11-30
2      2020-12-10
3      2020-12-22
4      2020-12-21
5             NaT
          ...    
7384          NaT
7385          NaT
7386          NaT
7388   2021-03-25
7389   2021-04-08
Name: date_of_test_kit_completion, Length: 6702, dtype: datetime64[ns]

In [ ]:
pd.to_datetime("21-01-25 00:00:00")

Timestamp('2025-01-21 00:00:00')

In [ ]:
all_surveys

NameError: name 'all_surveys' is not defined

In [ ]:
LABS_REGEX = "/projects/bdata/datasets/gatesfoundation/raw/COVID_Signals/*/all_other_datafiles/lab/lab_order_updates/*.parquet"
labs = vaex.open(LABS_REGEX).to_pandas_df()
consented_labs = labs[labs["id_participant_external"].astype(int).isin(set(public_participants))]
consented_labs.to_csv("/projects/bdata/datasets/gatesfoundation/processed/COVID_Signals/labs.csv")

In [ ]:
labs

,id_participant_external,samples_received,approval_date,status,result,native_order_id
0,424954,2020-12-30T00:44:51.028-05:00,2020-12-31T10:38:31.093-05:00,approved,Negative,195300
1,774964,2020-12-29T14:51:46.213-05:00,2020-12-31T11:13:32.818-05:00,approved,Negative,205705
2,307629,2020-12-30T12:09:16.513-05:00,2020-12-31T11:15:58.450-05:00,approved,Negative,188800
3,988260,2020-12-29T19:00:32.956-05:00,2020-12-31T10:28:03.078-05:00,approved,Positive,195632
4,248718,2020-12-30T12:52:09.374-05:00,2020-12-31T11:15:57.812-05:00,approved,Negative,186323
...,...,...,...,...,...,...
6564,935047,2021-04-08T11:33:30.646-04:00,2021-04-08T21:17:27.943-04:00,sent,Negative,208844
6565,935047,2021-04-01T10:32:11.476-04:00,2021-04-01T22:00:23.117-04:00,sent,Negative,208846
6566,290847,2021-03-26T10:48:33.437-04:00,2021-03-26T21:33:26.241-04:00,sent,Negative,234559
6567,290847,2021-03-26T10:48:33.437-04:00,2021-03-26T21:33:26.241-04:00,sent,Negative,234559


In [ ]:

self_labs = consented_labs
self_labs["date_received"]=pd.to_datetime(self_labs["samples_received"],utc=True).astype('<M8[ns]')

self_weekly_surveys = weekly_surveys_consented
self_weekly_surveys["date_completed"] = pd.to_datetime(weekly_surveys_consented["date_of_test_kit_completion"].str.split("|").str[-1],errors="coerce")
self_weekly_surveys = self_weekly_surveys.dropna(subset=["date_completed"])

/tmp/ipykernel_352837/4182606940.py:2: FutureWarning: Using .astype to convert from timezone-aware dtype to timezone-naive dtype is deprecated and will raise in a future version.  Use obj.tz_localize(None) or obj.tz_convert('UTC').tz_localize(None) instead
  self_labs["date_received"]=pd.to_datetime(self_labs["samples_received"],utc=True).astype('<M8[ns]')
/tmp/ipykernel_352837/4182606940.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self_labs["date_received"]=pd.to_datetime(self_labs["samples_received"],utc=True).astype('<M8[ns]')
/tmp/ipykernel_352837/4182606940.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

/tmp/ipykernel_352837/3057129168.py:1: FutureWarning: Using .astype to convert from timezone-aware dtype to timezone-naive dtype is deprecated and will raise in a future version.  Use obj.tz_localize(None) or obj.tz_convert('UTC').tz_localize(None) instead
  pd.to_datetime(self_labs["samples_received"],utc=True).astype('<M8[ns]')


0      2020-12-30 05:44:51.028
1      2020-12-29 19:51:46.213
2      2020-12-30 17:09:16.513
4      2020-12-30 17:52:09.374
5      2020-12-30 02:38:59.189
                 ...          
6564   2021-04-08 15:33:30.646
6565   2021-04-01 14:32:11.476
6566   2021-03-26 14:48:33.437
6567   2021-03-26 14:48:33.437
6568   2021-04-16 14:59:47.265
Name: samples_received, Length: 5975, dtype: datetime64[ns]

In [ ]:
results_and_surveys= pd.merge_asof(self_labs.sort_values("date_received"),
                                    self_weekly_surveys.sort_values("date_completed"),
                                    by="id_participant_external",
                                    left_on="date_received",
                                    right_on="date_completed",
                                    tolerance=pd.to_timedelta("7D"),
                                    direction="backward")

In [ ]:
results_and_surveys.columns

Index(['id_participant_external', 'samples_received', 'approval_date',
       'status', 'result', 'native_order_id', 'date_received',
       'survey_occurrence', 'response_status', 'tm_start',
       ...
       'number_of_hours_missed_due_to_health_problems',
       'number_of_hours_missed_due_to_vacation_holidays',
       'number_of_hours_actually_worked',
       'health_problems_affect_on_productivity__',
       'health_problems_affect_on_regular_daily_activities__',
       'past_week_felt__in_the_last_week_how_often_have_you_felt_difficulties_were_piling_up_so_high_that_you_could_not_overcome_them_',
       'past_week_felt__in_the_last_week_how_often_have_you_felt_that_you_were_unable_to_control_the_important_things_in_your_life_',
       'past_week_felt__in_the_last_week_how_often_have_you_felt_confident_about_your_ability_to_handle_your_personal_problems_',
       'past_week_felt__in_the_last_week_how_often_have_you_felt_that_things_were_going_your_way_',
       'date_completed'],

,date_completed,result
109,2020-11-23,Negative
110,2020-11-23,Negative
113,2020-11-23,Negative
120,2020-11-24,Negative
131,2020-11-23,Negative
...,...,...
5968,2021-04-07,Negative
5969,2021-04-06,Negative
5971,2021-04-07,Negative
5972,2021-04-08,Negative


## Reformatting the raw Garmin Data from Covid Signals:

In [ ]:
GARMIN_ACTIVITY_REGEX = "/projects/bdata/datasets/gatesfoundation/raw/COVID_Signals/*/all_other_datafiles/garmin/activities/*.parquet"
garmin_activity = vaex.open(GARMIN_ACTIVITY_REGEX)

In [ ]:
list(garmin_activity.columns)

['id_participant_external',
 'startTimeInSeconds',
 'startTimeOffsetInSeconds',
 'activityType',
 'durationInSeconds',
 'averageHeartRateInBeatsPerMinute',
 'averageRunCadenceInStepsPerMinute',
 'averageSpeedInMetersPerSecond',
 'averagePaceInMinutesPerKilometer',
 'distanceInMeters',
 'maxBikeCadenceInRoundsPerMinute',
 'maxHeartRateInBeatsPerMinute',
 'maxPaceInMinutesPerKilometer',
 'maxRunCadenceInStepsPerMinute',
 'maxSpeedInMetersPerSecond',
 'numberOfActiveLengths',
 'steps',
 'totalElevationGainInMeters',
 'totalElevationLossInMeters']

In [ ]:
garmin_activity["activityType"].value_counts()

WALKING                          9250
RUNNING                          4683
INDOOR_CARDIO                    1315
STRENGTH_TRAINING                 770
OTHER                             352
CYCLING                           294
TREADMILL_RUNNING                 252
YOGA                              238
ELLIPTICAL                        211
INDOOR_CYCLING                    125
VIRTUAL_RIDE                       85
ROAD_BIKING                        47
LAP_SWIMMING                       39
PILATES                            29
HIKING                             21
FITNESS_EQUIPMENT                  13
STAIR_CLIMBING                     11
INDOOR_ROWING                      10
TRAIL_RUNNING                       9
VIRTUAL_RUN                         9
WINTER_SPORTS                       4
RESORT_SKIING_SNOWBOARDING_WS       3
CASUAL_WALKING                      2
SNOW_SHOE_WS                        1
INDOOR_RUNNING                      1
RECUMBENT_CYCLING                   1
BREATHWORK  

In [ ]:
len(garmin_activity)

17789

In [ ]:
garmin_activity["id_participant_external"].nunique()

519

In [ ]:
garmin_activity.column_names

['id_participant_external',
 'startTimeInSeconds',
 'startTimeOffsetInSeconds',
 'activityType',
 'durationInSeconds',
 'averageHeartRateInBeatsPerMinute',
 'averageRunCadenceInStepsPerMinute',
 'averageSpeedInMetersPerSecond',
 'averagePaceInMinutesPerKilometer',
 'distanceInMeters',
 'maxBikeCadenceInRoundsPerMinute',
 'maxHeartRateInBeatsPerMinute',
 'maxPaceInMinutesPerKilometer',
 'maxRunCadenceInStepsPerMinute',
 'maxSpeedInMetersPerSecond',
 'numberOfActiveLengths',
 'steps',
 'totalElevationGainInMeters',
 'totalElevationLossInMeters']

In [ ]:
GARMIN_SLEEPS_REGEX = "/projects/bdata/datasets/gatesfoundation/raw/COVID_Signals/*/all_other_datafiles/garmin/sleeps/*.parquet"
garmin_sleeps = vaex.open(GARMIN_SLEEPS_REGEX)

In [ ]:
garmin_sleeps.column_names

['id_participant_external',
 'calendarDate',
 'startTimeInSeconds',
 'startTimeOffsetInSeconds',
 'durationInSeconds',
 'unmeasurableSleepInSeconds',
 'deepSleepDurationInSeconds',
 'lightSleepDurationInSeconds',
 'remSleepInSeconds',
 'awakeDurationInSeconds',
 'sleepLevelsMap',
 'validation',
 'timeOffsetSleepRespiration',
 'timeOffsetSleepSpo2']

In [ ]:
garmin_sleeps = garmin_sleeps.to_pandas_df()

In [ ]:
# Based off of the FitBit "classic" sleep api: https://dev.fitbit.com/build/reference/web-api/sleep/get-sleep-log-by-date/
GARMIN_SLEEP_STAGE_MAP={
    "light": 1,
    "deep":2,
    "rem":2,
    "awake":3
}

In [ ]:
garmin_sleeps["sleepLevelsMap"] = garmin_sleeps["sleepLevelsMap"].map(json.loads)

In [ ]:
exploded_sleep = (pd.melt(
    pd.concat([garmin_sleeps,pd.json_normalize(garmin_sleeps["sleepLevelsMap"])],axis=1),
    id_vars=["id_participant_external","startTimeOffsetInSeconds"],
    value_vars=["awake","light","rem","deep"],
    var_name="sleep_type"
    ).explode("value").dropna(subset=["value"])
)


In [ ]:
processed_sleep = pd.concat([exploded_sleep.drop(columns="value").reset_index(drop=True),pd.json_normalize(exploded_sleep["value"])],axis=1)

In [ ]:
processed_sleep["endTime"] = pd.to_datetime(processed_sleep["endTimeInSeconds"], unit="s") + pd.to_timedelta(processed_sleep["startTimeOffsetInSeconds"])
processed_sleep["startTime"] = pd.to_datetime(processed_sleep["startTimeInSeconds"], unit="s") + pd.to_timedelta(processed_sleep["startTimeOffsetInSeconds"])
processed_sleep["sleep_classic"] = processed_sleep["sleep_type"].map(GARMIN_SLEEP_STAGE_MAP) 
processed_sleep["id_participant_external"] = processed_sleep["id_participant_external"].astype("category") 
processed_sleep["durationInSeconds"] = (processed_sleep["endTime"] - processed_sleep["startTime"]).dt.seconds
processed_sleep["timestamp"] = processed_sleep["startTime"] 
processed_sleep = processed_sleep.dropna().set_index("id_participant_external")

In [ ]:
processed_sleep.head(10)

,startTimeOffsetInSeconds,sleep_type,endTimeInSeconds,startTimeInSeconds,endTime,startTime,sleep_classic,durationInSeconds,timestamp
id_participant_external,,,,,,,,,
108757,-21600,awake,1608625680,1608625320,2020-12-22 08:27:59.999978400,2020-12-22 08:21:59.999978400,3,360,2020-12-22 08:21:59.999978400
108757,-21600,awake,1608630420,1608630360,2020-12-22 09:46:59.999978400,2020-12-22 09:45:59.999978400,3,60,2020-12-22 09:45:59.999978400
108757,-21600,awake,1608630960,1608630540,2020-12-22 09:55:59.999978400,2020-12-22 09:48:59.999978400,3,420,2020-12-22 09:48:59.999978400
943285,-18000,awake,1608620220,1608619980,2020-12-22 06:56:59.999982000,2020-12-22 06:52:59.999982000,3,240,2020-12-22 06:52:59.999982000
682052,-18000,awake,1608636780,1608636720,2020-12-22 11:32:59.999982000,2020-12-22 11:31:59.999982000,3,60,2020-12-22 11:31:59.999982000
604242,-21600,awake,1608551340,1608551280,2020-12-21 11:48:59.999978400,2020-12-21 11:47:59.999978400,3,60,2020-12-21 11:47:59.999978400
604242,-21600,awake,1608551520,1608551460,2020-12-21 11:51:59.999978400,2020-12-21 11:50:59.999978400,3,60,2020-12-21 11:50:59.999978400
604242,-21600,awake,1608551940,1608551760,2020-12-21 11:58:59.999978400,2020-12-21 11:55:59.999978400,3,180,2020-12-21 11:55:59.999978400
604242,-21600,awake,1608552660,1608552600,2020-12-21 12:10:59.999978400,2020-12-21 12:09:59.999978400,3,60,2020-12-21 12:09:59.999978400


In [ ]:
processed_sleep["date"] = processed_sleep["endTime"].dt.date
len(processed_sleep.reset_index().drop_duplicates(["id_participant_external","date"]))

35328

In [ ]:
garmin_activity.column_names

['id_participant_external',
 'startTimeInSeconds',
 'startTimeOffsetInSeconds',
 'activityType',
 'durationInSeconds',
 'averageHeartRateInBeatsPerMinute',
 'averageRunCadenceInStepsPerMinute',
 'averageSpeedInMetersPerSecond',
 'averagePaceInMinutesPerKilometer',
 'distanceInMeters',
 'maxBikeCadenceInRoundsPerMinute',
 'maxHeartRateInBeatsPerMinute',
 'maxPaceInMinutesPerKilometer',
 'maxRunCadenceInStepsPerMinute',
 'maxSpeedInMetersPerSecond',
 'numberOfActiveLengths',
 'steps',
 'totalElevationGainInMeters',
 'totalElevationLossInMeters']

In [ ]:
processed_heart_rate = garmin_activity[["id_participant_external","startTimeInSeconds","startTimeOffsetInSeconds",
                                        'averageHeartRateInBeatsPerMinute',"durationInSeconds"]].to_pandas_df()
processed_heart_rate = processed_heart_rate.dropna(subset=["durationInSeconds"])                                        
processed_heart_rate["timestamp"] = pd.to_datetime(processed_heart_rate["startTimeInSeconds"],unit="s") +\
                                    pd.to_timedelta(processed_heart_rate["startTimeOffsetInSeconds"].astype(int), unit="s")
processed_heart_rate["durationInSeconds"] = processed_heart_rate["durationInSeconds"].astype(int)
processed_heart_rate["id_participant_external"] = processed_heart_rate["id_participant_external"].astype("category") 
processed_heart_rate = processed_heart_rate.dropna().set_index("id_participant_external")
processed_heart_rate.head(10)

,startTimeInSeconds,startTimeOffsetInSeconds,averageHeartRateInBeatsPerMinute,durationInSeconds,timestamp
id_participant_external,,,,,
139209,1608597574,-18000,134,735,2020-12-21 19:39:34
760451,1608556071,-21600,99,944,2020-12-21 07:07:51
139209,1608593758,-18000,107,555,2020-12-21 18:35:58
403728,1608565909,-21600,112,1934,2020-12-21 09:51:49
737470,1608546973,-18000,66,610,2020-12-21 05:36:13
831129,1608578714,-21600,102,1426,2020-12-21 13:25:14
917852,1608559880,-18000,69,953,2020-12-21 09:11:20
807214,1608614949,-25200,89,2711,2020-12-21 22:29:09
480902,1608595236,-21600,100,946,2020-12-21 18:00:36


In [ ]:
processed_steps = garmin_activity[["id_participant_external","startTimeInSeconds","startTimeOffsetInSeconds",
                                        'steps',"durationInSeconds"]].to_pandas_df()
processed_steps = processed_steps.dropna(subset=["durationInSeconds"])  
processed_steps["timestamp"] = pd.to_datetime(processed_steps["startTimeInSeconds"],unit="s") +\
                                    pd.to_timedelta(processed_steps["startTimeOffsetInSeconds"].astype(int), unit="s")
processed_steps["durationInSeconds"] = processed_steps["durationInSeconds"].astype(int)
processed_steps["id_participant_external"] = processed_steps["id_participant_external"].astype("category")
processed_steps = processed_steps.dropna().set_index("id_participant_external")
processed_steps.head(10)

,startTimeInSeconds,startTimeOffsetInSeconds,steps,durationInSeconds,timestamp
id_participant_external,,,,,
139209,1608597574,-18000,40,735,2020-12-21 19:39:34
760451,1608556071,-21600,1704,944,2020-12-21 07:07:51
403728,1608565909,-21600,10,1934,2020-12-21 09:51:49
831129,1608578714,-21600,1590,1426,2020-12-21 13:25:14
917852,1608559880,-18000,86,953,2020-12-21 09:11:20
480902,1608595236,-21600,1434,946,2020-12-21 18:00:36
350093,1608585841,-18000,2470,1546,2020-12-21 16:24:01
861847,1608552241,-18000,1510,813,2020-12-21 07:04:01
202028,1608583933,-21600,984,778,2020-12-21 14:52:13


In [ ]:
from src.data.make_dataset import explode_str_column, get_new_index, safe_loc
from src.data.utils import  process_minute_level_pandas
from tqdm import tqdm

In [28]:
processed_out_path = "/projects/bdata/datasets/gatesfoundation/processed/COVID_Signals/garmin_minute_level_activity_two"
users_with_steps = processed_steps.index.unique()
all_results = []

for user in tqdm(users_with_steps.values):
    exploded_sleep = explode_str_column(safe_loc(processed_sleep,user),
                                target_col = "sleep_classic",
                                # rename_target_column="sleep_classic",
                                start_col="startTime",
                                dur_col = "durationInSeconds",
                                single_val=True,
                                dtype=pd.Int8Dtype())
    exploded_hr =  explode_str_column(safe_loc(processed_heart_rate,user),
                                        target_col = "averageHeartRateInBeatsPerMinute",
                                        rename_target_column="heart_rate",
                                        start_col="timestamp",
                                        single_val=True,
                                        dur_col = "durationInSeconds",
                                        dtype=pd.Int8Dtype())
    exploded_steps = explode_str_column(safe_loc(processed_steps,user),
                                        target_col = "steps",
                                        # rename_target_column="averageHeartRateInBeatsPerMinute",
                                        start_col="timestamp",
                                        single_val=True,
                                        dur_col = "durationInSeconds",
                                        dtype=pd.Int8Dtype())
    steps_and_hr = exploded_steps.join(exploded_hr,how = "left") 
    merged = steps_and_hr.join(exploded_sleep,how="left")                        

    
    processed = process_minute_level_pandas(minute_level_df = merged)

    # Keep datatypes in check
    processed["heart_rate"] = processed["heart_rate"].astype(pd.Int16Dtype())
    processed["participant_id"] = user
    all_results.append(processed)

all_results = pd.concat(all_results)
all_results["sleep_classic_0"] = all_results["sleep_classic_0"].fillna(False)
all_results["sleep_classic_1"] = all_results["sleep_classic_1"].fillna(False)
all_results["sleep_classic_2"] = all_results["sleep_classic_2"].fillna(False)
all_results["sleep_classic_3"] = all_results["sleep_classic_3"].fillna(False)

all_results.to_parquet(path = processed_out_path, partition_cols=["date"], engine="fastparquet")

  0%|          | 0/505 [00:00<?, ?it/s]/homes/gws/mikeam/MobileSensingSuite/src/data/make_dataset.py:81: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  new_index = pd.date_range(start_ts,end_ts,freq=freq,closed="left").values


In [1]:
processed_heart_rate.dtypes

NameError: name 'processed_heart_rate' is not defined

In [ ]:
processed_sleep["timestamp"].median()

Timestamp('2021-01-23 07:51:59.999978496')